The next cell ensures that we can use the full width of the screen in each cell of our notebook.

In [1]:
from IPython.core.display import HTML
with open('../style.css', 'r') as file:
    css = file.read()
HTML(css)

# <a href="https://en.wikipedia.org/wiki/Zebra_Puzzle">The Zebra Puzzle</a>

The following puzzle appeared in the magazine *Life International* on the 17th of December in the year 1962:
<ol>
    <li>There are five houses.</li>
    <li>The Englishman lives in the red house.</li>
    <li>The Spaniard owns the dog.</li>
    <li>Coffee is drunk in the green house.</li>
    <li>The Ukrainian drinks tea.</li>
    <li>The green house is immediately to the right of the ivory house.</li>
    <li>The Old Gold smoker owns snails.</li>
    <li>Kools are smoked in the yellow house.</li>
    <li>Milk is drunk in the middle house.</li>
    <li>The Norwegian lives in the first house.</li>
    <li>The man who smokes Chesterfields lives in the house next to the man with the fox.</li>
    <li>Kools are smoked in the house next to the house where the horse is kept.</li>
    <li>The Lucky Strike smoker drinks orange juice.</li>
    <li>The Japanese smokes Parliaments.</li>
    <li>The Norwegian lives next to the blue house.</li>
</ol>
Furthermore, each of the five houses is painted in a different colour, their inhabitants are of different nationalities, own different pets, drink different beverages, and smoke different brands of cigarettes.

Your task is to write a program that answers the following questions: 
<ul>
    <li><b>Who drinks water?</b></li>
    <li><b>Who owns the zebra?</b></li>
</ul>

## Choosing the Appropriate Variables

In order to solve this problem we use the following propositional variables:  
* For $i \in \{1,\cdots,5\}$ the variable $\texttt{English}\langle i \rangle$ expresses the fact
  that the Englishman lives in house number $i$.
  The remaining nationalities are coded using the variables
  $\texttt{Spanish}\langle i \rangle$, $\texttt{Ukrainian}\langle i \rangle$, $\texttt{Norwegian}\langle i \rangle$,      
  and $\texttt{Japanese}\langle i \rangle$.
* For $i \in \{1,\cdots,5\}$ the variable $\mathtt{Red}\langle i \rangle$ expresses that house number $i$ is red.
  We use the variables $\texttt{Green}\langle i \rangle$, $\texttt{Ivory}\langle i \rangle$, $\texttt{Yellow}\langle i \rangle$, 
  and $\texttt{Blue}\langle i \rangle$ to encode the remaining colors.
* For $i \in \{1,\cdots,5\}$ the variable $\texttt{OldGold}\langle i \rangle$ expresses that the inhabitant of house      
  number $i$ smokes cigarettes of the brand "Old Gold".
  We use the variables $\texttt{Kools}\langle i \rangle$, $\texttt{Chesterfields}\langle i \rangle$,                      
  $\texttt{LuckyStrike}\langle i \rangle$, and $\texttt{Parliaments}\langle i \rangle$ to encode the remaining
  cigarette brands.
* For $i \in \{1,\cdots,5\}$ the variable $\texttt{Dog}\langle i \rangle$ expresses the fact that the inhabitant 
  of house number $i$ keeps a dog as his pet. We use the variables
  $\texttt{Snails}\langle i \rangle$, $\texttt{Fox}\langle i \rangle$, $\texttt{Horse}\langle i \rangle$, and
  $\texttt{Zebra}\langle i \rangle$ to encode the remaining pets.
* For $i \in \{1,\cdots,5\}$ the variable $\texttt{Coffee}\langle i \rangle$ expresses the fact the the 
  inhabitant of house number $i$ drinks coffee.  We use the variables
  $\texttt{Milk}\langle i \rangle$, $\texttt{OrangeJuice}\langle i \rangle$, $\texttt{Tea}\langle i \rangle$, 
  and $\texttt{Water}\langle i \rangle$ to encode the remaining drinks.

We are using the angular brackets "$\langle$" and "$\rangle$" because our parser for propositional logic accepts these symbols as part of variable names.  The parser would be confused if we would use the normal parenthesis.

## Importing the Necessary Modules

Our goal is to solve this puzzle by first coding it as a solvability problem of propositional logic and then to solve the resulting set of clauses using the algorithm of Davis and Putnam.

In [2]:
%%capture 
%run Davis-Putnam-JW.ipynb

In order to be able to transform formulas from propositional logic into sets of clauses we import the module <tt>cnf</tt> which implements the function <tt>normalize</tt> that takes a formula and transforms it into a set of clauses.

In [3]:
%%capture
%run CNF.ipynb

In order to write formulas conveniently, we use the parser for propositional logic.

In [4]:
import propLogParser as plp

Using the parser and the module <tt>cnf</tt> we can impement a function $\texttt{parseKNF}(s)$ that takes a string $s$ representing a formula and transforms $s$ into an equivalent set of clauses.

In [5]:
def parseKNF(s):
    nestedTuple = plp.LogicParser(s).parse()
    Clauses     = normalize(nestedTuple)
    return Clauses

In [6]:
parseKNF('p ∨ r → q')

{frozenset({('¬', 'p'), 'q'}), frozenset({('¬', 'r'), 'q'})}

## Auxiliary Functions

In order to succinctly express the constraints that all houses have different colours, the inhabitants have different nationalities etc., it is convenient to implement a function $\texttt{atMostOne}(V)$ that takes a set of variables $V$ and returns a set of formulas that is true if and only if all the variables from $V$ have different values.

In [7]:
def atMostOne(V):
    return { frozenset({('¬',p), ('¬', q)}) for p in V
                                            for q in V 
                                            if  p != q 
           }

Given a name $f$ and an index $i \in\{1,2,3,4,5\}$, the function $\texttt{var}(i)$ creates the string 
$f\langle i \rangle$, e.g. the call `var("Japanese", 2)` the following string:
```
Japanese<2>
```

In [8]:
def var(f, i):
    return f + "<" + str(i) + ">" 

In [9]:
var("Japanese", 2)

'Japanese<2>'

A call of the form $\texttt{somewhere}(x)$ will return a clause that specifies that the person with property $x$ has to live in one of the houses from $1$ to $5$.  In order to be able to insert this clause into a set, make sure that you return a <tt>frozenset</tt>.

In [10]:
def somewhere(x):
    return frozenset({ var(x, i) for i in range(1, 5+1) })

In [11]:
somewhere("a")

frozenset({'a<1>', 'a<2>', 'a<3>', 'a<4>', 'a<5>'})

Given an exclusive set of properties $S$ and a house number $i$, the function $\texttt{atMostOneAt}(S, i)$ returns a set of clauses that specifies that the person living in house number $i$ has at most one of the properties from the set $S$.  For example, if 
$S = \{\texttt{"Japanese"}, \texttt{"Englishman"}, \texttt{"Spaniard"}, \texttt{"Norwegian"}, \texttt{"Ukranian"}\}$, 
then $\texttt{atMostOneAt}(S, 3)$ specifies that the inhabitant of house number 3 has at most one of the nationalities from the set $S$.

In [12]:
def atMostOneAt(S, i):
    "your code here"

In [13]:
atMostOneAt({"A", "B", "C"}, 1)

Implement a function $\texttt{onePerHouse}(S)$ which could be called as follows:
$$\texttt{onePerHouse}(\{\texttt{"Japanese"},
       \texttt{"English"}, 
       \texttt{"Spanish"}, \texttt{"Norwegian"}, 
       \texttt{"Ukrainian"}\})
$$
This function would create a set of clauses that expresses that there has to be a house where the Japanese lives, a house where the Englishman lives, a house where the Spaniard lives, a house where the Norwegian lives, and a house
where the Ukranian lives.  Furthermore, the set of clauses would create clauses that express that these five persons live in **different** houses.

When implementing this function, remember that you can use both the function $\texttt{somewhere}$ and the function $\texttt{atMostOneAt}$.

In [14]:
def onePerHouse(S):
    "your code here"

In [15]:
onePerHouse({"A", "B"})

Given to properties $a$ and $b$ the function $\texttt{sameHouse}(a, b)$ computes a set of clauses that specifies that if the inhabitant of house number $i$ has the property $a$, then he also has the the property $b$ and vice versa.  For example, $\texttt{sameHouse}(\texttt{"Japanese"}, \texttt{"Dog"})$ specifies that the Japanese guy keeps a dog.

In [16]:
def sameHouse(a, b):
    "your code here"

In [17]:
sameHouse("Red", "Tea")

Given to properties $a$ and $b$ the function $\texttt{nextTo}(a, b)$ computes a set of clauses that specifies that the inhabitants with properties $a$ and $b$ are direct neighbours.  For example, $\texttt{nextTo}(\texttt{'Japanese'}, \texttt{'Dog'})$ specifies that the Japanese guy lives next to the guy who keeps a dog.

In [18]:
def nextTo(a, b):
    "your code here"

In [19]:
nextTo('A', 'B')

Given to properties $a$ and $b$ the function $\texttt{leftTo}(a, b)$ computes a list of clauses that specifies that the inhabitants with properties $a$ lives in the house to the left of the inhabitant who has property $b$.  For example, $\texttt{livesTo}(\texttt{'Japanese'}, \texttt{'Dog'})$ specifies that the Japanese guy lives in the house to the left of the house where there is a dog.

In [20]:
def leftTo(a, b):
    "your code here"

In [21]:
leftTo('A', 'B')

In [22]:
Nations  = { "English", "Spanish", "Ukrainian", "Norwegian", "Japanese" }
Drinks   = { "Coffee", "Tea", "Milk", "OrangeJuice", "Water" }
Pets     = { "Dog", "Snails", "Horse", "Fox", "Zebra" }
Brands   = { "LuckyStrike", "Parliaments", "Kools", "Chesterfields", "OldGold" }
Colours  = { "Red", "Green", "Ivory", "Yellow", "Blue" }

In [23]:
def allClauses():
    # Every house has exactly one inhabitant.  This inhabitant has exactly one
    # nationality, one pet, smokes one brand of cigarettes, and has one type
    # of drink.  Furthermore, every house has exactly one color.
    "your code here"
    # The Englishman lives in the red house.
    Clauses |= sameHouse("English", "Red")
    # The Spaniard owns the dog.
    Clauses |= sameHouse("Spanish", "Dog")
    # Coffee is drunk in the green house.
    Clauses |= sameHouse("Coffee", "Green")
    # The Ukrainian drinks tea.
    Clauses |= sameHouse("Ukrainian", "Tea")
    # The green house is immediately to the right of the ivory house.
    Clauses |= leftTo("Ivory", "Green")
    # The Old Gold smoker owns snails.
    Clauses |= sameHouse("OldGold", "Snails")
    # Kools are smoked in the yellow house.
    Clauses |= sameHouse("Kools", "Yellow")
    # Milk is drunk in the middle house.
    Clauses |= parseKNF("Milk<3>")
    # The Norwegian lives in the first house.
    Clauses |= parseKNF("Norwegian<1>")
    # The man who smokes Chesterfields lives in the house next 
    # to the man with the fox.
    Clauses |= nextTo("Chesterfields", "Fox")
    # Kools are smoked in the house next to the house where the horse is kept.
    Clauses |= nextTo("Kools", "Horse")
    # The Lucky Strike smoker drinks orange juice.
    Clauses |= sameHouse("LuckyStrike", "OrangeJuice")
    # The Japanese smokes Parliaments.
    Clauses |= sameHouse("Japanese", "Parliaments")
    # The Norwegian lives next to the blue house.
    Clauses |= nextTo("Norwegian", "Blue")
    return Clauses

In [24]:
Clauses = allClauses()
Clauses

UnboundLocalError: local variable 'Clauses' referenced before assignment

In [ ]:
len(Clauses)

In [ ]:
def main():
    Clauses = allClauses()
    return solve(Clauses, set())

Solving the problem takes about 0.1 seconds on my computer.

In [ ]:
%%time
Solution = main()

## Functions to PrettyPrint the Solution

In [ ]:
def arb(S):
    for x in S:
        return x

In [ ]:
def extractAssignment(Solution):
    Assignment = {}
    for Unit in Solution:
        Literal = arb(Unit)
        if isinstance(Literal, str):
            number = int(Literal[-2])
            name   = Literal[:-3]
            Assignment[name] = number
    return Assignment

In [ ]:
extractAssignment(Solution)

In [ ]:
def showHTML(Solution):
    Assignment = extractAssignment(Solution)
    result  = '<table style="border:2px solid blue">\n'
    result += '<tr>'
    for name in ['House', 'Nationality',  'Drink', 'Animal', 'Brand', 'Colour']:
        result += '<th style="color:gold; background-color:blue">' + name + '</th>'
    result += '</tr>\n'
    for chair in range(1, 5+1):
        result += '<tr><td style="border:1px solid green">' + str(chair) + '</td>'
        for Class in [Nations, Drinks, Pets, Brands, Colours]:
            for x in Class:
                if Assignment[x] == chair:
                    result += '<td  style="border:1px solid green">' + x + '</td>'
        result += '</tr>\n'
    result += '</table>'
    display(HTML(result))

In [ ]:
showHTML(Solution)

## Checking the Uniqueness of the Solution

Given a set of unit clauses $U$, the function $\texttt{negateSolution}(U)$ returns a clause that is the logical negation of $U$.

In [ ]:
def negateSolution(UnitClauses):
    return { complement(arb(unit)) for unit in UnitClauses }

In [ ]:
negateSolution({frozenset({'a'}), frozenset({('¬', 'b')})})

In [ ]:
def checkUniqueness(Solution, Clauses):
    negation = negateSolution(Solution)
    Clauses.add(frozenset(negation))
    alternative = solve(Clauses)
    if alternative == { frozenset() }:
        print("Well done: The solution is unique!")
    else:
        print("ERROR: The solution is not unique!")

In [ ]:
%%time
checkUniqueness(Solution, Clauses)